In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from time import time
from sklearn.utils import shuffle
import os
tf.reset_default_graph()
data_path="D:\\log\\kaggle\\PetImages"
tf_data_path="D:\\log\\kaggle\\TFRecordData"
def CreateTFRecordData(data_path,tf_data_path):
    print("Create TFRecord:")
    if not os.path.exists(tf_data_path):
        os.makedirs(tf_data_path)
    if not os.path.exists(os.path.join(tf_data_path,"train_data")):
        os.makedirs(os.path.join(tf_data_path,"train_data"))
    if not os.path.exists(os.path.join(tf_data_path,"test_data")):
        os.makedirs(os.path.join(tf_data_path,"test_data"))
    train_writer=tf.python_io.TFRecordWriter(os.path.join(tf_data_path,"train_data\\train.TFRecord"))
    test_writer=tf.python_io.TFRecordWriter(os.path.join(tf_data_path,"test_data\\test.TFRecords"))
    classes=os.listdir(data_path)
    image_train_path=[]
    image_train_label=[]
    image_test_path=[]
    image_test_label=[]
    for label,class_name in enumerate(classes):
        class_path=os.path.join(data_path,class_name)
        image_index=0
        for image_name in os.listdir(class_path):
            image_index+=1
            image_path=os.path.join(class_path,image_name)
            if image_index<=12000:
                image_train_path.append(image_path)
                image_train_label.append(label)
            else:
                image_test_path.append(image_path)
                image_test_label.append(label)
    image_train_path,image_train_label=shuffle(image_train_path,image_train_label)
    image_test_path,image_test_label=shuffle(image_test_path,image_test_label)
    image_train_path,image_train_label=shuffle(image_train_path,image_train_label)
    image_test_path,image_test_label=shuffle(image_test_path,image_test_label)
    for i in range(len(image_train_path)):
        if (i+1)%2000==0:
            print("creating train_data TFRecord file:%.3f%%"%(100*i/len(image_train_path)))
        image=Image.open(image_train_path[i]).resize((150,150))
        image=image.convert("RGB").tobytes()
        example=tf.train.Example(features=tf.train.Features(feature={
                    "image":tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
                    "label":tf.train.Feature(int64_list=tf.train.Int64List(value=[image_train_label[i]]))
                }))
        train_writer.write(example.SerializeToString())
    for i in range(len(image_test_path)):
        if (i+1)%200==0:
            print("creating train_data TFRecord file:%.3f%%"%(100*i/len(image_test_path)))
        image=Image.open(image_test_path[i]).resize((150,150))
        image=image.convert("RGB").tobytes()
        example=tf.train.Example(features=tf.train.Features(feature={
                    "image":tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
                    "label":tf.train.Feature(int64_list=tf.train.Int64List(value=[image_test_label[i]]))
                }))
        test_writer.write(example.SerializeToString())
    train_writer.close()
    test_writer.close()

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from time import time
from sklearn.utils import shuffle
import os
tf.reset_default_graph()
data_path="D:\\log\\kaggle\\PetImages"
tf_data_path="D:\\log\\kaggle\\TFRecordData"
trained_model_path='D:\\log\\kaggle\\trained_model_path_AlexNet_kaggle'
name=["cat","dog"]
def imshows(images,labels,index,amount,predictions=None):
    fig=plt.gcf()
    fig.set_size_inches(10,20)
    for i in range(amount):
        title="lab:"+name[np.argmax(labels[index+i])]
        if predictions is not None:
            title=title+"prd:"+name[np.argmax(predictions[index+i])]
        ax=plt.subplot(5,6,i+1)
        ax.set_title(title)
        ax.imshow(images[index+i])
    plt.show()
def Batcher(name,batch_size,enhance=False,path=tf_data_path,min_after_dequeue=1500,num_threads=2):
    path=os.path.join(tf_data_path,name)
    filename=os.listdir(os.path.join(tf_data_path,name))
    for i in range(len(filename)):
        filename[i]=os.path.join(path,filename[i])
        print("find data file:",filename[i])
    filename_queue=tf.train.string_input_producer(filename,shuffle=False,num_epochs=None)
    reader=tf.TFRecordReader()
    _,serialized_examples=reader.read(filename_queue)
    features=tf.parse_single_example(serialized_examples,features={
            "image":tf.FixedLenFeature([],tf.string),
            "label":tf.FixedLenFeature([],tf.int64)
        })
    image=tf.decode_raw(features["image"],tf.uint8)
    label=tf.cast(features["label"],tf.int32)
    image=tf.reshape(image,[150,150,3])
    image=tf.random_crop(image,[128,128,3])
    if enhance:
        image=tf.image.random_flip_left_right(image)
        image=tf.image.random_flip_up_down(image)
        image=tf.image.random_brightness(image,max_delta=0.4)
        image=tf.image.random_contrast(image,lower=0.7,upper=1.3)
        image=tf.image.random_saturation(image,lower=0.7,upper=1.3)
    image=tf.image.per_image_standardization(image)
    images,labels=tf.train.shuffle_batch([image,label],
                                         num_threads=num_threads,
                                         batch_size=batch_size,
                                         min_after_dequeue=min_after_dequeue,
                                         capacity=min_after_dequeue+batch_size*3)
    return images,tf.one_hot(labels,2)
#CreateTFRecordData(data_path,tf_data_path)
image_train,label_train=Batcher(name="train_data",batch_size=256,enhance=True)
image_test,label_test=Batcher(name="test_data",batch_size=256,enhance=False)

find data file: D:\log\kaggle\TFRecordData\train_data\train.TFRecord
find data file: D:\log\kaggle\TFRecordData\test_data\test.TFRecords


In [ ]:
def exponential_decay_with_warmup(warmup_step,learning_rate_base,global_step,learning_rate_step,learning_rate_decay,staircase=False):
    warmup_step=tf.constant(warmup_step)
    linear_increase=learning_rate_base*tf.cast(global_step/warmup_step,tf.float32)
    exponential_decay=tf.train.exponential_decay(learning_rate_base,
                                                                  global_step-warmup_step,
                                                                  learning_rate_step,
                                                                  learning_rate_decay,
                                                                  staircase=staircase)
    learning_rate=tf.cond(global_step<=warmup_step,lambda:linear_increase,lambda:exponential_decay)
    return learning_rate
class CNN():
    def __init__(self,drop_rate=0.0,is_training=False,regularizer=None,regularizer_name='losses',average_class=None):
        self.layer_index=1
        self.drop_rate=drop_rate
        self.is_training=is_training
        self.Rname=regularizer_name
        if regularizer is None:
            self.regularizer=None
        else:
            self.regularizer=regularizer
        if average_class is None:
            self.ema=None
        else:
            self.ema=average_class.average
    def Xavier(self): #initializer
        return tf.contrib.layers.xavier_initializer()
    def MSRA(self,shape): # initializer
        if len(shape)==2:
            return tf.truncated_normal_initializer(stddev=2/shape[1])
        else:
            return tf.truncated_normal_initializer(stddev=2/(shape[0]*shape[1]*shape[2]))
    def Sigmoid(self,tensor): #activation
        with tf.variable_scope("L%d_Sigmoid"%self.layer_index,reuse=tf.AUTO_REUSE):
            self.layer_index+=1
            return tf.nn.sigmoid(tensor)   
    def Relu(self,tensor):  #activation
        with tf.variable_scope("L%d_Relu"%self.layer_index,reuse=tf.AUTO_REUSE):
            self.layer_index+=1
            return tf.nn.relu(tensor)
    def Conv2d(self,tensor,output_channel,ksize,strides,padding="VALID"):
        with tf.variable_scope("L%d_Conv2d"%self.layer_index,reuse=tf.AUTO_REUSE):
            self.layer_index+=1
            input_channel=tensor.shape.as_list()[-1]
            kernel_shape=[ksize,ksize,input_channel,output_channel]
            kernel=tf.get_variable("kernel",kernel_shape,initializer=self.Xavier())
            bias=tf.get_variable("bias",[output_channel],initializer=tf.constant_initializer(0.0))
            if self.ema is not None: 
                conv2d=tf.nn.conv2d(tensor,self.ema(kernel),strides=[1,strides,strides,1],padding=padding)
                bias_add=tf.nn.bias_add(conv2d,self.ema(bias))
            else:
                conv2d=tf.nn.conv2d(tensor,kernel,strides=[1,strides,strides,1],padding=padding)
                bias_add=tf.nn.bias_add(conv2d,bias)
            return bias_add
    def MaxPooling2d(self,tensor,ksize,strides,padding="VALID"):
        with tf.variable_scope("L%d_MaxPooling2d"%self.layer_index,reuse=tf.AUTO_REUSE):
            self.layer_index+=1
            return tf.nn.max_pool(tensor,ksize=[1,ksize,ksize,1],strides=[1,strides,strides,1],padding=padding)
    def BatchNormalization(self,tensor):
        with tf.variable_scope("L%d_BatchNorm"%self.layer_index,reuse=tf.AUTO_REUSE):
            self.layer_index+=1
            return tf.layers.batch_normalization(tensor,training=self.is_training)
    def Dropout(self,tensor,drop_rate):
        with tf.variable_scope("L%d_Dropout"%self.layer_index,reuse=tf.AUTO_REUSE):
            self.layer_index+=1
        return tf.layers.dropout(tensor,drop_rate,training=self.is_training)
    def GlobalAvgPooling2d(self,tensor):
        with tf.variable_scope("L%d_GlobalAvgPooling2d"%self.layer_index,reuse=tf.AUTO_REUSE):
            self.layer_index+=1
            tensor_shape=tensor.shape.as_list()
            ksize=[1,tensor_shape[1],tensor_shape[2],1]
            return tf.nn.avg_pool(tensor,ksize=ksize,strides=[1,1,1,1],padding="VALID")
    def FC_conv2d(self,tensor,output_channel):  #1x1卷积代替全连接层
        with tf.variable_scope("L%d_FullCon"%self.layer_index,reuse=tf.AUTO_REUSE):
            self.layer_index+=1
            return self.Conv2d(tensor,output_channel=output_channel,ksize=1,strides=1,padding="SAME")
    def SE_Block(self,tensor,SE_rate=16): #Sequeeze and Excitation Block,or SENet,which is a subNet
         with tf.variable_scope("L%d_SqueezeAndExcitation"%self.layer_index,reuse=tf.AUTO_REUSE):
            self.layer_index+=1       
            input_channel=tensor.shape.as_list()[-1]
            squeeze=self.GlobalAvgPooling2d(tensor)
            excitation=self.FC_conv2d(squeeze,input_channel/SE_rate)
            BN=self.BatchNormalization(excitation)
            relu=self.Relu(BN)
            excitation=self.FC_conv2d(relu,input_channel)
            sigmoid=self.Sigmoid(excitation)
            return sigmoid*tensor
    def AlexNet(self,tensor):
        conv=self.Conv2d(tensor,output_channel=96,ksize=11,strides=4,padding="VALID")
        BN=self.BatchNormalization(conv)
        relu=self.Relu(BN)
        
        SE=self.SE_Block(relu)
        
        pool=self.MaxPooling2d(SE,ksize=3,strides=2,padding="VALID")
        
        conv=self.Conv2d(pool,output_channel=256,ksize=5,strides=2,padding="SAME")
        BN=self.BatchNormalization(conv)
        relu=self.Relu(BN)
        
        SE=self.SE_Block(relu)
        
        
        conv=self.Conv2d(SE,output_channel=384,ksize=3,strides=1,padding="SAME")
        BN=self.BatchNormalization(conv)
        relu=self.Relu(BN)
        
        SE=self.SE_Block(relu)
        
        conv=self.Conv2d(SE,output_channel=384,ksize=3,strides=1,padding="SAME")
        BN=self.BatchNormalization(conv)
        relu=self.Relu(BN)
        
        SE=self.SE_Block(relu)
        
        conv=self.Conv2d(SE,output_channel=256,ksize=3,strides=2,padding="SAME")
        BN=self.BatchNormalization(conv)
        relu=self.Relu(BN)
        SE=self.SE_Block(relu)
        
        
        Global_avg_pool=self.GlobalAvgPooling2d(SE)
        BN=self.BatchNormalization(Global_avg_pool)
        
        full=self.FC_conv2d(BN,784)
        BN=self.BatchNormalization(full)
        relu=self.Relu(BN)
        dropout=self.Dropout(relu,drop_rate=self.drop_rate)
        
        full=self.FC_conv2d(BN,128)
        BN=self.BatchNormalization(full)
        relu=self.Relu(BN)
        dropout=self.Dropout(relu,drop_rate=self.drop_rate)
        
        full=self.FC_conv2d(dropout,2)
        return tf.reshape(full,[-1,2])
        
x=tf.placeholder(tf.float32,[None,128,128,3])
y=tf.placeholder(tf.float32,[None,2])
is_training=tf.placeholder(tf.bool)
train_epoches=60
batch_size=150
batch_num=int(24000/batch_size)
#预热的指数衰减学习率
global_step=tf.Variable(0,trainable=False)
warmup_step=batch_num*3
learning_rate_base=0.015
learning_rate_decay=0.95
learning_rate_step=batch_num
learning_rate=exponential_decay_with_warmup(warmup_step,learning_rate_base,global_step,learning_rate_step,learning_rate_decay)
#正则化和前向预测
regularizer=tf.contrib.layers.l2_regularizer(0.008)
forward=CNN(drop_rate=0.5,is_training=is_training,regularizer=regularizer,average_class=None).AlexNet(x)
#滑动平均
moving_average_decay=0.999
ema=tf.train.ExponentialMovingAverage(moving_average_decay,global_step)
ema_op=ema.apply(tf.trainable_variables())
ema_forward=CNN(drop_rate=0.2,is_training=is_training,regularizer=None,average_class=ema).AlexNet(x)
prediction=tf.nn.softmax(ema_forward)
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
#损失函数和优化器
momentum=0.9
cross_entropy=tf.losses.softmax_cross_entropy(onehot_labels=y,logits=forward,label_smoothing=0.1)
l2_regularizer_loss=tf.add_n(tf.get_collection("losses"))
loss_function=cross_entropy+l2_regularizer_loss
update_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer=tf.train.MomentumOptimizer(learning_rate,momentum).minimize(loss_function,global_step=global_step)
if not os.path.exists(trained_model_path):
    os.makedirs(trained_model_path)
saver=tf.train.Saver(tf.global_variables())

In [ ]:
#开始进行训练
import tensorflow as tf
from PIL import Image
acc_1_list=[0.0]
loss_1_list=[0.0]
acc=0.0
loss=0.0
step=0    
start_time=time()
image_path="D:\\log\\kaggle\\4.jpg"
image=Image.open(image_path)
std_image=image.resize((128,128))
std_image=std_image.convert("RGB")
std_image=tf.image.per_image_standardization(std_image)
std_image=tf.reshape(std_image,[1,128,128,3])
with tf.Session() as sess:
    trained_model_state=tf.train.get_checkpoint_state(trained_model_path)
    if trained_model_state and trained_model_state.model_checkpoint_path:
        saver.restore(sess,trained_model_state.model_checkpoint_path)    
    sess.run([tf.local_variables_initializer(),tf.global_variables_initializer()])
    coord=tf.train.Coordinator()
    threads=tf.train.start_queue_runners(sess=sess,coord=coord)
    print("training start!")
    for epoch in range(0,train_epoches):
        for batch in range(batch_num):
            
            images,labels=sess.run([image_train,label_train])
            sess.run([optimizer,ema_op],feed_dict={x:images,y:labels,is_training:True})
            print(batch,sess.run(accuracy,feed_dict={x:images,y:labels,is_training:True}))
            
        images,labels=sess.run([image_test,label_test])
        loss,acc=sess.run([loss_function,accuracy],feed_dict={x:images,y:labels,is_training:False})
        decay=min(0.9,(1+step)/(10+step))
        print("Epoch:%d Speed:%.2f seconds per epoch -> Accuracy:%.3f  Loss:%.3f"%(epoch+1,(time()-start_time)/(1+epoch),acc,loss))
        print("learning rate:%.7f "%(sess.run(learning_rate)))
        saver.save(sess,os.path.join(trained_model_path,"trained_model_%d"%(epoch+1)))
        print("model_%d saved"%(epoch+1))
#        input_image=sess.run(std_image)
#        pred=sess.run(prediction,feed_dict={x:input_image,is_training:False})
#        plt.imshow(image)
#        p=100*pred[0][1]
#        plt.title("The probability of a dog is:%.2f%%"%(p))
#        plt.show()
        
    print("training finished after %.2f seconds"%(time()-start_time))
    coord.request_stop()
    coord.join(threads)

INFO:tensorflow:Restoring parameters from D:\log\kaggle\trained_model_path_AlexNet_kaggle\trained_model_37
training start!
0 0.53125
1 0.484375
2 0.48828125
3 0.54296875
4 0.5234375
5 0.51171875
6 0.58203125
7 0.5703125
8 0.58984375
9 0.55859375
10 0.5859375
11 0.5390625
12 0.5390625
13 0.6171875
14 0.609375
15 0.55859375
16 0.63671875
17 0.55859375
18 0.53125
19 0.5390625
20 0.54296875
21 0.5234375
22 0.57421875
23 0.57421875
24 0.578125
25 0.5625
26 0.609375
27 0.59375
28 0.6015625
29 0.6015625
30 0.62890625
31 0.625
32 0.58203125
33 0.5703125
34 0.61328125
35 0.578125
36 0.58984375
37 0.60546875
38 0.58203125
39 0.6171875
40 0.61328125
41 0.6015625
42 0.60546875
43 0.5625
44 0.57421875
45 0.60546875
46 0.609375
47 0.578125
48 0.62109375
49 0.5625
50 0.58203125
51 0.62890625
52 0.64453125
53 0.66796875
54 0.6328125
55 0.67578125
56 0.64453125
57 0.671875
58 0.64453125
59 0.609375
60 0.58984375
61 0.6015625
62 0.62109375
63 0.65234375
64 0.640625
65 0.6328125
66 0.6015625
67 0.6640625

In [ ]:
plt.xlabel('Step')
plt.ylabel('Average_Accuracy')
plt.plot(acc_1_list[20:])
plt.legend(['SE-AlexNet'],loc='lower right')
plt.show()


plt.xlabel('Step')
plt.ylabel('Average_Loss')
plt.plot(loss_1_list[20:])
plt.legend(['SE-AlexNet'],loc='upper right')
plt.show()

In [ ]:
import tensorflow as tf
from PIL import Image
image_path="D:\\log\\kaggle\\3.jpg"
#image_path="D:\\log\\kaggle\\PetImages\\Dog\\4.jpg"
image=Image.open(image_path)
std_image=image.resize((180,180))
std_image=std_image.convert("RGB")
std_image=tf.random_crop(std_image,[128,128,3])
std_image=tf.image.per_image_standardization(std_image)
std_image=tf.reshape(std_image,[1,128,128,3])
print("waiting")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    trained_model_state=tf.train.get_checkpoint_state(trained_model_path)
    if trained_model_state and trained_model_state.model_checkpoint_path:
        saver.restore(sess,trained_model_state.model_checkpoint_path)
    input_image=sess.run(std_image)
    pred=sess.run(prediction,feed_dict={x:input_image,is_training:False})
    for i in range(9):
        pred+=sess.run(prediction,feed_dict={x:input_image,is_training:False})
    pred=sess.run(tf.nn.softmax(pred))
    plt.imshow(image)
    if pred[0][0]>pred[0][1]:
        plt.title("The probability of a cat is:%.2f%%"%(100*pred[0][0]))
    else:
        plt.title("The probability of a dog is:%.2f%%"%(100*pred[0][1]))
    plt.show()
        

In [ ]:
with tf.Session() as sess:
    sess.run([tf.local_variables_initializer(),tf.global_variables_initializer()])
    coord=tf.train.Coordinator()
    threads=tf.train.start_queue_runners(sess=sess,coord=coord)
    img,lab=sess.run([image_test,label_test])
    imshows(img,lab,0,30)
    print(lab)
    coord.request_stop()
    coord.join(threads)